# 1 Regular Languages

----
## 1.1 Finite Automata

단수 automaton 복수 automata

유한오토마타(finite automata, FA)를 유한상태기계(finite state machine, FSM)이라고도 함

### Formal definition of a finite automaton

### Examples of finite automata

### Designing finite automata

### The regular operations

----
## 1.2 Nondeterminism

### Formal defintion of a nondeterministic finite automata

### Equivalenceof NFAs and DFAs

### Closure under the regular operations

----
## 1.3 Regular Expression

### Formal definition of a regular expression

### Equivalence with finite automata

----
## 1.4 Nonregular Languages

### The pumping lemma for regular languages